In [1]:
import time
import math
import uuid
from datetime import datetime, timedelta, timezone

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import joblib
import talib
import ta
import ccxt  # rea

In [2]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [3]:
fig = None
price_trace = None
buy_order_trace = None
sell_order_trace = None
long_pos_trace = None
short_pos_trace = None
grid_lines = []
grid_traces = []

def init_plot():
    global fig, price_trace, long_pos_trace, short_pos_trace, grid_traces

    # --- Price line trace (empty at start) ---
    price_trace = go.Scatter(
        x=[], y=[],
        mode="lines",
        line=dict(color="cyan", width=2),
        name="BTC/USDT Price"
    )

    # --- Position markers ---
    long_pos_trace = go.Scatter(
        x=[], y=[], mode="markers",
        marker=dict(color="green", size=10, symbol="triangle-up"),
        name="Long Entries"
    )
    short_pos_trace = go.Scatter(
        x=[], y=[], mode="markers",
        marker=dict(color="red", size=10, symbol="triangle-down"),
        name="Short Entries"
    )

    fig = go.Figure()
    fig.add_trace(price_trace)
    fig.add_trace(long_pos_trace)
    fig.add_trace(short_pos_trace)

    fig.update_layout(
        title="BTC/USDT Grid Bot Live",
        xaxis_title="Time", yaxis_title="Price",
        template="plotly_dark"
    )

    fig.show()
    grid_traces = []




In [4]:
symbol = "BTC/USDT"
market_type = "spot"           # 'spot' implemented here. For futures you'd change code.
model_timeframe = "15m"
price_timeframe = "1s"
limit = 300
window_size = 10

FEATURES = [
    'rsi', 'ema12', 'ema26', 'macd', 'signal', 'histogram',
    'dema9', 'sma3', 'tsi', '%k', '%d',
    'atr', 'obv', 'adx', 'cci',
    'return_1', 'roll_mean_5', 'roll_std_5'
]
SAVED_MODEL = "greg_tech_8.pth"
SCALER_FILE = "scaler_15m.pkl"

min_tp_pct = 0.01
atr_period = 14
atr_multiplier = 0.1
position_fraction = 0.05
reduced_fraction = 0.5
grid_levels = 16
grid_spacing_atr_factor = 0.1
max_total_exposure = 0.7

prediction_interval = 15 * 60
check_interval_seconds = 1
simulate_only = True  # KEEP TRUE while testing
hold_factor = 0.2

# Forced cutoff parameters for frozen grids
frozen_timeout_seconds = 60 * 60  # if a frozen grid still has open positions/orders after 1 hour -> forced cancel
frozen_max_drawdown_pct = 0.2     # if grid drawdown > 20% of its allocated USD -> force close

# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [5]:
# ===============================
# 5. Define LSTM Model
# ===============================
class CryptoLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(CryptoLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, (hn, cn) = self.lstm(x)
        out = out[:, -1, :]  # last timestep
        out = self.fc(out)
        return out

input_dim = len(FEATURES)
hidden_dim = 128
num_layers = 4
output_dim = 3

model = CryptoLSTM(input_dim, hidden_dim, num_layers, output_dim).to(device)
model.load_state_dict(torch.load(SAVED_MODEL, map_location=device))
model.eval()
scaler = joblib.load(SCALER_FILE)

/home/gregory/Trading_bot/venv/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
"""# -------------------------
# MODEL
# -------------------------
class CryptoLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=4, output_dim=3):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        return self.fc(out)

model = CryptoLSTM(input_dim=len(FEATURES), hidden_dim=128, num_layers=4, output_dim=3)
model.load_state_dict(torch.load(SAVED_MODEL, map_location=device))
model.to(device); model.eval()
scaler = joblib.load(SCALER_FILE)
"""

'# -------------------------\n# MODEL\n# -------------------------\nclass CryptoLSTM(nn.Module):\n    def __init__(self, input_dim, hidden_dim=128, num_layers=4, output_dim=3):\n        super().__init__()\n        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)\n        self.fc = nn.Linear(hidden_dim, output_dim)\n    def forward(self, x):\n        out, _ = self.lstm(x)\n        out = out[:, -1, :]\n        return self.fc(out)\n\nmodel = CryptoLSTM(input_dim=len(FEATURES), hidden_dim=128, num_layers=4, output_dim=3)\nmodel.load_state_dict(torch.load(SAVED_MODEL, map_location=device))\nmodel.to(device); model.eval()\nscaler = joblib.load(SCALER_FILE)\n'

In [7]:
# -------------------------
# CCXT EXCHANGE SETUP
# -------------------------
exchange = ccxt.binance({
    'enableRateLimit': True,
})

In [8]:
# Helper: fetch minimums / step sizes for rounding
MARKET_INFO = {}
def load_market_info():
    global MARKET_INFO
    markets = exchange.load_markets()
    if symbol in markets:
        info = markets[symbol]
        MARKET_INFO['precision'] = info.get('precision', {})
        MARKET_INFO['limits'] = info.get('limits', {})
    else:
        MARKET_INFO['precision'] = {}
        MARKET_INFO['limits'] = {}

load_market_info()

In [9]:
def compute_tsi(close, r1=25, r2=13):
    delta = close.diff()
    ema1 = delta.ewm(span=r1, adjust=False).mean()
    ema2 = ema1.ewm(span=r2, adjust=False).mean()

    abs_delta = delta.abs()
    abs_ema1 = abs_delta.ewm(span=r1, adjust=False).mean()
    abs_ema2 = abs_ema1.ewm(span=r2, adjust=False).mean()

    tsi = 100 * (ema2 / abs_ema2)
    return tsi


In [10]:
import ta
import talib

def compute_indicators(df):
    """
    Compute a rich set of technical indicators for ML models.
    Works with DataFrames containing: 'open', 'high', 'low', 'close', 'volume'.
    Returns: DataFrame of features aligned with input df index.
    """
    df = df.copy()
    df.columns = [c.lower() for c in df.columns]

    out = {}

    # --- Core Indicators ---
    out["RSI"] = ta.momentum.RSIIndicator(df["close"], window=14).rsi()
    out["EMA12"] = df["close"].ewm(span=12, adjust=False).mean()
    out["EMA26"] = df["close"].ewm(span=26, adjust=False).mean()
    out["MACD"] = out["EMA12"] - out["EMA26"]
    out["Signal"] = out["MACD"].ewm(span=9, adjust=False).mean()
    out["Histogram"] = out["MACD"] - out["Signal"]

    # --- DEMA (with fallback) ---
    try:
        out["DEMA9"] = talib.DEMA(df["close"].values, timeperiod=9)
    except Exception:
        out["DEMA9"] = df["close"].ewm(span=9, adjust=False).mean()

    # --- SMA ---
    out["SMA3"] = ta.trend.sma_indicator(df["close"], window=3)

    # --- True Strength Index (TSI) ---
    def compute_tsi(close, r1=25, r2=13):
        delta = close.diff()
        ema1 = delta.ewm(span=r1, adjust=False).mean()
        ema2 = ema1.ewm(span=r2, adjust=False).mean()
        abs_delta = delta.abs()
        abs_ema1 = abs_delta.ewm(span=r1, adjust=False).mean()
        abs_ema2 = abs_ema1.ewm(span=r2, adjust=False).mean()
        tsi = 100 * (ema2 / (abs_ema2.replace(0, np.nan)))
        return tsi.fillna(0)

    out["TSI"] = compute_tsi(df["close"])

    # --- Stochastic Oscillator (%K, %D) ---
    period, smooth_k, smooth_d = 14, 3, 3
    lowest_low = df["low"].rolling(period).min()
    highest_high = df["high"].rolling(period).max()
    out["%K"] = 100 * (df["close"] - lowest_low) / (highest_high - lowest_low + 1e-8)
    out["%K"] = out["%K"].rolling(smooth_k).mean()
    out["%D"] = out["%K"].rolling(smooth_d).mean()

    # --- Advanced Indicators ---
    out["ATR"] = ta.volatility.AverageTrueRange(
        df["high"], df["low"], df["close"], window=14
    ).average_true_range()

    out["OBV"] = ta.volume.OnBalanceVolumeIndicator(
        df["close"], df["volume"]
    ).on_balance_volume()

    out["ADX"] = ta.trend.ADXIndicator(
        df["high"], df["low"], df["close"], window=14
    ).adx()

    out["CCI"] = ta.trend.cci(
        df["high"], df["low"], df["close"], window=20
    )

    # --- Price-derived stats ---
    out["Return_1"] = df["close"].pct_change()
    out["Roll_Mean_5"] = df["close"].rolling(5).mean()
    out["Roll_Std_5"] = df["close"].rolling(5).std()

    # Convert all to DataFrame
    feat_df = pd.DataFrame(out, index=df.index)
    feat_df = feat_df.dropna().reset_index(drop=True)
    feat_df.columns = [c.lower() for c in feat_df.columns]

    return feat_df

In [11]:
# -------------------------
# FETCH DATA
# -------------------------
def fetch_latest_ohlcv(symbol, timeframe, limit):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp','open','high','low','close','volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df.columns = [c.lower() for c in df.columns]
    return df


In [12]:
def get_current_atr():
    df = fetch_latest_ohlcv(symbol, model_timeframe, atr_period + 2)
    df['prev_close'] = df['close'].shift(1)
    df['TR'] = df.apply(lambda r: max(r['high'] - r['low'],
                                      abs(r['high'] - r['prev_close']),
                                      abs(r['low'] - r['prev_close'])), axis=1)
    df['ATR'] = df['TR'].rolling(atr_period).mean()
    return float(df['ATR'].iloc[-1])


In [13]:
print(get_current_atr())

415.3257142857141


In [14]:
# -------------------------
# Prediction
# -------------------------
def predict_label_from_window(window_features):
    flat = scaler.transform(window_features)
    x = torch.tensor(flat, dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
        probs[1] *= hold_factor
        probs = probs / probs.sum()
        pred = int(np.argmax(probs))
    return pred  # 0=down 1=hold 2=up


In [15]:
# -------------------------
# Trading state & helpers
# -------------------------
balance = 100.0
open_positions = []   # {id, side, entry, qty, size_usd, grid_id, ccxt_order_id (if placed)}
open_orders = []      # {id, grid_id, side, price, size_usd, status, ccxt_id}
grids = {}            # grid metadata
def usd_to_qty(usd, price):
    # naive qty = usd / price; later round using market precision
    return usd / price

def round_amount_qty(qty):
    prec = MARKET_INFO.get('precision', {}).get('amount')
    if prec is None:
        return qty
    prec = int(prec)  # ensure integer
    return float(round(qty, prec))

def round_price(price):
    prec = MARKET_INFO.get('precision', {}).get('price')
    if prec is None:
        return price
    prec = int(prec)  # ensure integer
    return float(round(price, prec))


In [16]:
# -------------------------
# Real order helpers (ccxt)
# -------------------------
def place_limit_order_ccxt(side, price, size_usd):
    """
    Place a limit order (spot) for symbol.
    side: 'buy' or 'sell'
    price: limit price (float)
    size_usd: notional USD to spend (float)
    returns: dict with ccxt order info or raises
    """
    # derive amount (base asset) from USD notional
    price = float(price)
    amount = size_usd / price
    amount = round_amount_qty(amount)
    price = round_price(price)
    if amount <= 0:
        raise ValueError("Amount <= 0 after rounding; increase size_usd or check precision.")
    params = {}
    try:
        if simulate_only:
            # simulate by returning a fake ccxt order id
            return {'id': 'SIM-' + str(uuid.uuid4())[:8], 'status': 'open', 'price': price, 'amount': amount}
        else:
            order = exchange.create_limit_order(symbol, side, amount, price, params)
            return order
    except Exception as e:
        print("[CCXT PLACE ORDER ERROR]", e)
        raise

def fetch_order_ccxt(ccxt_order_id):
    if simulate_only:
        # simulated: search in open_orders
        for o in open_orders:
            if o.get('ccxt_id') == ccxt_order_id:
                return {'id': ccxt_order_id, 'status': o['status'], 'price': o['price']}
        return None
    else:
        try:
            return exchange.fetch_order(ccxt_order_id, symbol)
        except Exception as e:
            print("[CCXT FETCH ORDER ERROR]", e)
            return None

def cancel_order_ccxt(ccxt_order_id):
    if simulate_only:
        # mark simulated order as cancelled in open_orders
        for o in open_orders:
            if o.get('ccxt_id') == ccxt_order_id and o['status'] == 'open':
                o['status'] = 'cancelled'
                return True
        return False
    else:
        try:
            exchange.cancel_order(ccxt_order_id, symbol)
            return True
        except Exception as e:
            print("[CCXT CANCEL ERROR]", e)
            return False

def market_close_position_ccxt(pos):
    """
    For spot longs -> market sell to close.
    """
    if simulate_only:
        # calculate pnl and remove pos
        pnl = pos['qty'] * (pos['last_price'] - pos['entry'])
        return {'status': 'closed_sim', 'pnl': pnl}
    else:
        try:
            amount = round_amount_qty(pos['qty'])
            order = exchange.create_market_sell_order(symbol, amount)
            return order
        except Exception as e:
            print("[CCXT MARKET CLOSE ERROR]", e)
            return None


In [17]:
active_orders = []

In [18]:
def recreate_grid_order(price, side):
    """
    After a TP/SL close, recreate the grid order at the same level
    so it can trade again next time price returns there.
    """
    global open_orders, grids

    # find grid that matches this price (approx)
    grid_id = None
    for gid, meta in grids.items():
        for oid in meta.get('orders', []):
            for o in open_orders:
                if abs(o['price'] - price) < 1e-8 and o['grid_id'] == gid:
                    grid_id = gid
                    break

    # if no grid found (edge case), pick any active one
    if grid_id is None and len(grids) > 0:
        grid_id = list(grids.keys())[0]

    atr_now = get_current_atr()
    spacing = atr_now * grid_spacing_atr_factor
    size_usd = max(10.0, balance * position_fraction)

    # recreate same buy order
    if side == 'long':
        tp = price + spacing
        sl = price - 6 * spacing
        order = {
            'id': str(uuid.uuid4())[:10],
            'grid_id': grid_id,
            'side': 'buy',
            'price': float(price),
            'size_usd': float(size_usd),
            'status': 'open',
            'tp': float(tp),
            'sl': float(sl)
        }
    else:
        tp = price - spacing
        sl = price + 6 * spacing
        order = {
            'id': str(uuid.uuid4())[:10],
            'grid_id': grid_id,
            'side': 'sell',
            'price': float(price),
            'size_usd': float(size_usd),
            'status': 'open',
            'tp': float(tp),
            'sl': float(sl)
        }

    # add to global open_orders + grid record
    open_orders.append(order)
    if grid_id:
        grids[grid_id]['orders'].append(order['id'])

    print(f"[GRID REUSED] {side.upper()} order re-created at {price:.2f} (tp={tp:.2f}, sl={sl:.2f})")


In [19]:
# -------------------------
# Place grid with ccxt order creation
# -------------------------
def place_grid(grid_direction, center_price, mode='trend', fraction=position_fraction):
    """
    Symmetric ladder grid system:
    - Buy orders placed above and below center_price
    - Each grid acts as a buy point
    - TP = next higher grid price
    - SL = 6 grids below entry price
    """
    gid = str(uuid.uuid4())[:8]
    grids[gid] = {
        'id': gid,
        'direction': grid_direction,
        'active': True,
        'created_at': datetime.utcnow(),
        'orders': [],
        'frozen': False
    }

    atr_now = get_current_atr()
    spacing = atr_now * grid_spacing_atr_factor
    size_usd = max(10.0, balance * fraction)

    # Create symmetric grid levels
    half = grid_levels // 2
    grid_prices = [center_price + i * spacing for i in range(-half, half + 1)]
    grid_prices = sorted(set(grid_prices))  # ensure sorted, remove duplicates

    for i, buy_price in enumerate(grid_prices):
        # TP = next higher grid
        if i + 1 < len(grid_prices):
            tp_price = grid_prices[i + 1]
        else:
            tp_price = buy_price * (1 + min_tp_pct)

        # SL = 6 grids below
        sl_index = max(0, i - 6)
        sl_price = grid_prices[sl_index] if i - 6 >= 0 else buy_price - 6 * spacing

        try:
            resp = place_limit_order_ccxt('buy', buy_price, size_usd)
            ccxt_id = resp.get('id')
        except Exception:
            ccxt_id = None

        oid = str(uuid.uuid4())[:10]
        order = {
            'id': oid,
            'grid_id': gid,
            'side': 'buy',
            'price': float(buy_price),
            'size_usd': float(size_usd),
            'status': 'open',
            'ccxt_id': ccxt_id,
            'tp': float(tp_price),
            'sl': float(sl_price)
        }
        open_orders.append(order)
        grids[gid]['orders'].append(oid)

    print(f"[GRID PLACED] id={gid}, dir={grid_direction}, mode={mode}, levels={len(grid_prices)}")

    # Debug print: show all grid levels and their TP/SL
    print("------------------------------------------------------------")
    print(f"[DEBUG] Grid Details for {gid}:")
    for i, buy_price in enumerate(grid_prices):
        if i + 1 < len(grid_prices):
            tp_price = grid_prices[i + 1]
        else:
            tp_price = buy_price * (1 + min_tp_pct)
        sl_index = max(0, i - 6)
        sl_price = grid_prices[sl_index] if i - 6 >= 0 else buy_price - 6 * spacing
        print(f"Grid {i+1}/{len(grid_prices)} | Buy: {buy_price:.2f} | TP: {tp_price:.2f} | SL: {sl_price:.2f}")
    print("------------------------------------------------------------")

    return gid


def freeze_grid(gid):
    if gid in grids:
        grids[gid]['frozen'] = True
        grids[gid]['active'] = False
        grids[gid]['frozen_at'] = datetime.now(timezone.utc)
        print(f"[GRID FROZEN] id={gid}")

def cancel_grid_orders_ccxt(gid):
    global open_orders
    to_cancel = [o for o in open_orders if o['grid_id'] == gid and o['status'] == 'open']
    for o in to_cancel:
        ccid = o.get('ccxt_id')
        if ccid:
            cancel_order_ccxt(ccid)
        o['status'] = 'cancelled'
    print(f"[GRID CANCELLED] id={gid}, canceled_orders={len(to_cancel)}")
    grids.pop(gid, None)

def get_open_orders_for_grid(gid):
    return [o for o in open_orders if o['grid_id'] == gid and o['status'] == 'open']

def total_open_exposure():
    pos_exposure = sum([p['size_usd'] for p in open_positions])
    orders_exposure = sum([o['size_usd'] for o in open_orders if o['status'] == 'open'])
    return pos_exposure + orders_exposure


In [20]:
def has_position_from_grid(gid):
    """Return True if this grid currently has an active long position."""
    return any(p.get('grid_id') == gid and p.get('active', True) for p in open_positions)

def get_next_higher_price(entry_price):
    """
    Look through currently open buy orders and return the smallest price that is > entry_price.
    If none found, fallback to entry*(1+min_tp_pct).
    """
    buy_prices = sorted([o['price'] for o in open_orders if o['status']=='open' and o['side']=='buy'])
    for p in buy_prices:
        if p > entry_price:
            return p
    # fallback
    return entry_price * (1 + min_tp_pct)


In [21]:
def find_swings(df, length=9):
    """
    Identify swing highs and lows exactly like TradingView's ta.highest()/lowest() confirmation.
    """
    swing_highs = []
    swing_lows = []

    for i in range(len(df)):
        if i < length or i > len(df) - length - 1:
            swing_highs.append(False)
            swing_lows.append(False)
            continue

        # TV logic: candle is swing high if its high is the highest in [i-length, i]
        window_high = df['high'].iloc[i - length:i + 1].max()
        window_low = df['low'].iloc[i - length:i + 1].min()

        swing_highs.append(df['high'].iloc[i] == window_high)
        swing_lows.append(df['low'].iloc[i] == window_low)

    df['swing_high'] = swing_highs
    df['swing_low'] = swing_lows
    return df


In [22]:
def detect_order_blocks_and_msb(df, length=9, fib_factor=0.33):
    """
    Full TradingView-style MSB + OB detector.
    Returns bullish/bearish OB zones and MSB direction.
    """
    df = find_swings(df.copy(), length)
    ob_zones = {'bullish_ob': None, 'bearish_ob': None}
    msb = 'none'
    trend = None

    last_swing_high = None
    last_swing_low = None

    for i in range(len(df)):
        # Record new swings
        if df['swing_high'].iloc[i]:
            last_swing_high = df['high'].iloc[i]
        if df['swing_low'].iloc[i]:
            last_swing_low = df['low'].iloc[i]

        # Skip until we have both
        if last_swing_high is None or last_swing_low is None:
            continue

        # Check bullish MSB (close above last swing high)
        if df['close'].iloc[i] > last_swing_high:
            msb = 'bullish'
            trend = 1
            # Bullish OB: last bearish candle before breakout
            prev_red = df.iloc[:i][ (df.iloc[:i]['close'] < df.iloc[:i]['open']) ]
            if not prev_red.empty:
                red_idx = prev_red.index[-1]
                ob_zones['bullish_ob'] = (
                    df.loc[red_idx, 'low'],
                    df.loc[red_idx, 'high']
                )

        # Check bearish MSB (close below last swing low)
        elif df['close'].iloc[i] < last_swing_low:
            msb = 'bearish'
            trend = -1
            # Bearish OB: last bullish candle before breakdown
            prev_green = df.iloc[:i][ (df.iloc[:i]['close'] < df.iloc[:i]['open']) ]
            if not prev_green.empty:
                green_idx = prev_green.index[-1]
                ob_zones['bearish_ob'] = (
                    df.loc[green_idx, 'low'],
                    df.loc[green_idx, 'high']
                )

    return {'bullish_ob': ob_zones['bullish_ob'],
            'bearish_ob': ob_zones['bearish_ob'],
            'msb': msb,
            'trend': trend}


In [23]:
def generate_trade_signal(symbol, model_timeframe, limit, window_size=10):
    """
    Combines ML prediction + TV-style MSB/OB structure.
    """
    df = fetch_latest_ohlcv(symbol, model_timeframe, limit)
    ind_df = compute_indicators(df).dropna()
    feat_df = df.join(ind_df, how='inner').dropna()

    ml_signal = predict_label_from_window(feat_df[FEATURES].values[-window_size:])
    ob_msb = detect_order_blocks_and_msb(feat_df)

    last_close = float(df['close'].iloc[-1])

    # BUY logic
    if ml_signal == 2:  # uptrend
        if ob_msb['bearish_ob']:
            low, high = ob_msb['bearish_ob']
            if low <= last_close <= high:
                return "buy"
        elif ob_msb['msb'] == 'bullish':
            return "buy"

    # SELL logic
    elif ml_signal == 0:  # downtrend
        if ob_msb['bullish_ob']:
            low, high = ob_msb['bullish_ob']
            if low <= last_close <= high:
                return "sell"
        elif ob_msb['msb'] == 'bearish':
            return "sell"

    return "hold"


In [24]:
import plotly.graph_objects as go
from datetime import datetime, timezone

# Keep a global figure so it updates continuously
fig = go.Figure()
price_times = []
price_values = []
grid_traces = []
long_pos_trace = go.Scatter(mode='markers', marker=dict(color='green', size=10), name='Long')
short_pos_trace = go.Scatter(mode='markers', marker=dict(color='red', size=10), name='Short')

def update_plot(last_price, current_time, open_orders, open_positions):
    import plotly.graph_objects as go

    fig = go.Figure()
    
    # --- Price line ---
    price_times = [current_time]
    price_values = [last_price]
    fig.add_trace(go.Scatter(
        x=price_times,
        y=price_values,
        mode='lines+markers',
        line=dict(color='blue', width=2),
        name='Price'
    ))
    
    # --- Long/Short positions ---
    long_pos_x = [p['created_at'] for p in open_positions if p['side']=='long']
    long_pos_y = [p['entry'] for p in open_positions if p['side']=='long']
    short_pos_x = [p['created_at'] for p in open_positions if p['side']=='short']
    short_pos_y = [p['entry'] for p in open_positions if p['side']=='short']

    fig.add_trace(go.Scatter(
        x=long_pos_x,
        y=long_pos_y,
        mode='markers',
        marker=dict(color='green', size=10),
        name='Long Positions'
    ))

    fig.add_trace(go.Scatter(
        x=short_pos_x,
        y=short_pos_y,
        mode='markers',
        marker=dict(color='red', size=10),
        name='Short Positions'
    ))

    # --- Grid lines (from open_orders) ---
    grid_prices = [o['price'] for o in open_orders if o['status']=='open']
    for price in grid_prices:
        fig.add_trace(go.Scatter(
            x=[current_time - timedelta(minutes=30), current_time + timedelta(minutes=30)],  # extend line
            y=[price, price],
            mode='lines',
            line=dict(color='red', width=1, dash='dot'),
            name=f"Grid {price:.2f}"
        ))
    
    fig.update_layout(
        title="Price + Positions + Grids",
        xaxis_title="Time",
        yaxis_title="Price",
        showlegend=True
    )
    
    fig.show()


In [25]:
# -------------------------
# Execution simulation -> replaced by actual order monitoring
# -------------------------
def try_update_order_statuses():
    """
    Poll ccxt for open orders and update statuses in open_orders.
    For simulated orders (simulate_only) we keep status as is until filled via price checks.
    """
    if simulate_only:
        return
    # fetch open orders for symbol
    try:
        live_open = exchange.fetch_open_orders(symbol)
        live_ids = {o['id'] for o in live_open}
        # mark orders that are no longer open as 'filled' or 'closed'
        for o in open_orders:
            ccid = o.get('ccxt_id')
            if not ccid:
                continue
            if ccid not in live_ids and o['status'] == 'open':
                # fetch order detail
                det = fetch_order_ccxt(ccid)
                if det:
                    status = det.get('status')
                    o['status'] = status
    except Exception as e:
        print("[CCXT POLL ERROR]", e)

def try_fill_orders_sim_or_real(current_price):
    """
    Simulation: fill open limit orders when price touches them.

    Behavior:
    - On buy fill: create a long position, reserve the USD (balance -= size_usd),
      compute qty (rounded), store tp/sl taken from the order (order['tp'], order['sl']).
    - On sell fill (hold-mode sell): create a short position (kept for completeness).
    """
    global balance

    filled = []
    if simulate_only:
        for o in list(open_orders):
            if o['status'] != 'open':
                continue
            
            grid_id = o['grid_id']
            grid_price = o['price']
            
            # find this grid's lower and upper neighbors (for defining the zone)
            sorted_prices = sorted([oo['price'] for oo in open_orders if oo['grid_id'] == grid_id])
            i = sorted_prices.index(grid_price)

            lower_bound = sorted_prices[i - 1] if i > 0 else grid_price * 0.995  # 0.5% safety below
            upper_bound = sorted_prices[i + 1] if i + 1 < len(sorted_prices) else grid_price * 1.005
            
            next_prices = [p for p in sorted_prices if p > grid_price]
            tp_price = next_prices[0] if next_prices else grid_price * (1 + min_tp_pct)
            
            # check if zone already has a position
            zone_occupied = any(
                p['grid_id'] == grid_id and lower_bound < p['entry'] <= upper_bound
                for p in open_positions
            )
            if zone_occupied:
                continue  # skip creating duplicate position in this zone


            # ----- BUY fills (long) -----
            if o['side'] == 'buy' and lower_bound < current_price <= grid_price:
                
                size_usd = float(o['size_usd'])
                qty = usd_to_qty(size_usd, o['price'])
                # use rounding helper to avoid tiny qty becoming zero or wrong
                balance -= size_usd
                

                # create the position using the order's TP and SL (from the grid)
                pos = {
                    'id': str(uuid.uuid4())[:10],
                    'side': 'long',
                    'entry': float(o['price']),
                    'qty': qty,
                    'size_usd': size_usd,
                    'grid_id': o['grid_id'],
                    'created_at': datetime.now(timezone.utc),
                    'last_price': current_price,
                    'active': True,
                    # take TP/SL directly from the order placed by place_grid
                    'tp_price': float(o['tp']),
                    'sl_price': float(o['sl']),
                }

                open_positions.append(pos)
                o['status'] = 'filled'
                filled.append(o)
                print(f"[SIM FILL BUY] order={o['id']} price={o['price']:.2f} qty={qty:.6f} size_usd={size_usd:.2f} tp={pos['tp_price']:.2f} sl={pos['sl_price']:.2f}")

            # ----- SELL fills (hold-mode sells) -----
            elif o['side'] == 'sell' and current_price >= o['price']:
                size_usd = float(o['size_usd'])
                qty = usd_to_qty(size_usd, o['price'])
                
                pos = {
                    'id': str(uuid.uuid4())[:10],
                    'side': 'short',
                    'entry': float(o['price']),
                    'qty': qty,
                    'size_usd': size_usd,
                    'grid_id': o['grid_id'],
                    'created_at': datetime.now(timezone.utc),
                    'last_price': current_price,
                    'active': True,
                    'tp_price': upper_bound,
                    'sl_price': float(o['sl']),
                }
                # Shorts not primary; we don't reserve USD in this spot-sim
                open_positions.append(pos)
                o['status'] = 'filled'
                filled.append(o)
                print(f"[SIM FILL SELL] order={o['id']} price={o['price']:.2f} qty={qty:.6f} size_usd={size_usd:.2f} tp={pos['tp_price']:.2f} sl={pos['sl_price']:.2f}")
    else:
        # real mode: poll CCXT and reconcile
        try_update_order_statuses()
        for o in list(open_orders):
            if o['status'] not in ('open', None) and o['status'] != 'open':
                # when CCXT indicates fill, create positions from trade details (not implemented here)
                pass

    return filled



def try_resolve_positions_by_tp_sl(current_price):
    """
    Resolve positions by TP and SL using zone-based logic:
    - TP/SL are set based on neighboring grid levels at fill-time.
    - Positions exit only when price reaches the next zone.
    """
    global balance
    closed = []

    for pos in list(open_positions):
        grid_id = pos['grid_id']
        entry_price = pos['entry']

        # get sorted grid prices for this grid
        grid_prices = sorted([o['price'] for o in open_orders if o['grid_id'] == grid_id])

        # fallback if grid_orders cleared
        tp = pos.get('tp_price')
        sl = pos.get('sl_price')
        
        
        if pos['side'] == 'long':
            # Take Profit
            if current_price >= tp:
                exit_price = tp
                revenue = pos['qty'] * exit_price
                pnl = revenue - pos['size_usd']
                balance += revenue
                open_positions.remove(pos)
                closed.append((pos, 'TP', exit_price, pnl))
                print(f"[POS TP] pos={pos['id']} pnl={pnl:.2f} tp={exit_price:.2f} entry={entry_price:.2f} current={current_price:.2f}")
                recreate_grid_order(pos['entry'], pos['side'])

            # Stop Loss
            elif current_price <= sl:
                exit_price = current_price
                revenue = pos['qty'] * exit_price
                pnl = revenue - pos['size_usd']
                balance += revenue
                open_positions.remove(pos)
                closed.append((pos, 'SL', exit_price, pnl))
                print(f"[POS SL] pos={pos['id']} pnl={pnl:.2f} sl={sl:.2f} entry={entry_price:.2f} current={current_price:.2f}")
                recreate_grid_order(pos['entry'], pos['side'])

        elif pos['side'] == 'short':
            # TP for shorts (price drops to lower zone)
            if current_price <= tp:
                exit_price = tp
                revenue = pos['qty'] * (entry_price - tp)  # simplified
                pnl = revenue
                balance += pnl
                open_positions.remove(pos)
                closed.append((pos, 'TP_SHORT', tp, pnl))
                print(f"[POS TP SHORT] pos={pos['id']} pnl={pnl:.2f} entry={entry_price:.2f} current={current_price:.2f}")

            # SL for shorts (price rises to upper zone)
            elif current_price >= sl:
                exit_price = current_price
                pnl = pos['qty'] * (entry_price - current_price)
                balance += pnl
                open_positions.remove(pos)
                closed.append((pos, 'SL_SHORT', exit_price, pnl))
                print(f"[POS SL SHORT] pos={pos['id']} pnl={pnl:.2f} entry={entry_price:.2f} current={current_price:.2f}")

    return closed


In [26]:
# -------------------------
# Graceful transition utilities with forced cutoff
# -------------------------
def begin_graceful_transition(new_prediction, center_price):
    # freeze active grids
    active_gids = [gid for gid, meta in grids.items() if not meta.get('frozen', False)]
    for gid in active_gids:
        freeze_grid(gid)

    # set fraction reduced if frozen grids have exposure
    any_open_in_frozen = any([len(get_open_orders_for_grid(gid)) > 0 or any(p['grid_id']==gid for p in open_positions) for gid in active_gids])
    fraction = position_fraction * (reduced_fraction if any_open_in_frozen else 1.0)

    dir_map = {2: 'up', 0: 'down', 1: 'hold'}
    direction = dir_map.get(new_prediction, 'hold')
    new_gid = place_grid(direction, center_price, mode='trend', fraction=fraction)
    print(f"[TRANSITION] started new grid {new_gid} dir={direction} fraction={fraction:.3f}")
    return new_gid

def check_frozen_cutoffs(current_price):
    """
    For each frozen grid, check timeout and drawdown.
    If exceeded -> cancel all orders and close positions realistically
    (simulate selling at current market price).
    """
    global balance

    for gid, meta in list(grids.items()):
        if not meta.get('frozen', False):
            continue

        frozen_at = meta.get('frozen_at', datetime.now(timezone.utc))
        age = (datetime.now(timezone.utc) - frozen_at).total_seconds()

        # compute exposure and notional allocated to this grid
        orders = get_open_orders_for_grid(gid)
        exposure = sum([o['size_usd'] for o in orders])

        # if grid has no exposure or positions, remove it
        if exposure <= 0 and not any(p['grid_id'] == gid for p in open_positions):
            print(f"[CLEANUP] frozen grid {gid} cleared (no exposure).")
            grids.pop(gid, None)
            continue

        # timeout reached → force close everything
        if age > frozen_timeout_seconds:
            print(f"[FORCED CUTOFF] grid {gid} timed out ({age:.0f}s) → closing all positions at market and cancelling orders.")

            # 1️⃣ Cancel all open orders for this grid
            for o in list(open_orders):
                if o['grid_id'] == gid and o['status'] == 'open':
                    ccid = o.get('ccxt_id')
                    if ccid:
                        cancel_order_ccxt(ccid)
                    o['status'] = 'cancelled'

            # 2️⃣ Close all positions for this grid realistically
            for p in list(open_positions):
                if p['grid_id'] == gid:
                    # simulate closing at current market price
                    
                    exit_price = current_price

                    # compute revenue and pnl
                    revenue = p['qty'] * exit_price
                    pnl = revenue - p['size_usd']

                    # add revenue to balance
                    balance += revenue

                    print(f"[FORCED CLOSE] pos={p['id']} grid={gid} side={p['side']} entry={p['entry']:.2f} exit={exit_price:.2f} pnl={pnl:.2f}")

                    # remove from open positions
                    open_positions.remove(p)

            # 3️⃣ Remove grid metadata after closure
            grids.pop(gid, None)
            print(f"[GRID REMOVED] {gid} after forced cutoff.\n")

        # Optional: add drawdown-based cutoff logic later


In [27]:
def close_all_positions(current_price):
    """
    Force-close all positions in any grid (active or frozen) at current market price.
    Cancel all open orders and add revenue to balance.
    """
    global balance

    for gid, meta in list(grids.items()):
        # 1️⃣ Cancel all open orders for this grid
        for o in list(open_orders):
            if o['grid_id'] == gid and o['status'] == 'open':
                ccid = o.get('ccxt_id')
                if ccid:
                    cancel_order_ccxt(ccid)
                o['status'] = 'cancelled'

        # 2️⃣ Close all positions realistically
        for p in list(open_positions):
            if p['grid_id'] == gid:
                exit_price = current_price
                revenue = p['qty'] * exit_price
                pnl = revenue - p['size_usd']
                balance += revenue

                print(f"[FORCED CLOSE] pos={p['id']} grid={gid} side={p['side']} entry={p['entry']:.2f} exit={exit_price:.2f} pnl={pnl:.2f}")

                open_positions.remove(p)

        # 3️⃣ Remove grid metadata after closure
        grids.pop(gid, None)
        print(f"[GRID REMOVED] {gid} after forced closure.\n")


In [28]:
# Store initial capital
initial_balance = 100.0  # your real starting capital

def compute_realistic_profit(current_price):
    """
    Simulate closing all open positions right now.
    Returns:
        final_balance_if_stopped (float)
        net_profit (float)
    """
    # Start with balance + USD reserved in all open positions
    # This ensures we account for both free funds and funds already used
    total_reserved = sum(p['size_usd'] for p in open_positions)
    simulated_balance = balance + total_reserved

    # Add/subtract unrealized PnL for each open position
    for pos in open_positions:
        if pos['side'] == 'long':
            pnl = (current_price - pos['entry']) * pos['qty']
        else:  # short
            pnl = (pos['entry'] - current_price) * pos['qty']
        simulated_balance += pnl  # already added size_usd, so just add profit/loss

    # net profit compared to initial starting balance
    net_profit = simulated_balance - initial_balance
    return simulated_balance, net_profit


In [ ]:
# -------------------------
# Main run loop
# -------------------------
def run_bot():
    global balance
    last_prediction = None
    last_prediction_time = None

    # -----------------------------
    # Warm-up: initial prediction
    # -----------------------------
    back_df = fetch_latest_ohlcv(symbol, model_timeframe, 30 + window_size)
    feat_df = compute_indicators(back_df).dropna()

    # Run short back-prediction to determine recent trend
    preds = [
        predict_label_from_window(feat_df[FEATURES].values[i - window_size:i])
        for i in range(window_size, len(feat_df))
    ]

    # Use last non-hold prediction if available
    last_reversal = next((p for p in reversed(preds) if p != 1), None)
    last_prediction = last_reversal if last_reversal is not None else 1
    last_prediction_time = datetime.now(timezone.utc)

    # Fetch current market price
    last_price = float(fetch_latest_ohlcv(symbol, price_timeframe, 2)['close'].values[-1])

    # -----------------------------
    # Initial action based on first prediction
    # -----------------------------
    print(f"[INIT] last_pred={last_prediction} ({'DOWN' if last_prediction==0 else 'HOLD' if last_prediction==1 else 'UP'}), price={last_price:.2f}")

    if last_prediction == 0:
        # Start in paused mode — no grids placed
        print("[INIT STATUS] Prediction = DOWN → bot will start in PAUSED mode. Waiting for HOLD or UP...")
    else:
        # Start active grid trading
        mode_dir = 'up' if last_prediction == 2 else 'hold'
        print(f"[INIT STATUS] Starting grid trading in {mode_dir.upper()} mode.")
        place_grid(mode_dir, last_price, mode='trend')

    try:
        while True:
            now = datetime.now(timezone.utc)
            do_predict = (last_prediction_time is None) or ((now - last_prediction_time).total_seconds() >= prediction_interval)
            if do_predict:
                df_feat = fetch_latest_ohlcv(symbol, model_timeframe, limit)
                feat_df = compute_indicators(df_feat).dropna()
                if len(feat_df) >= window_size:
                    window = feat_df[FEATURES].values[-window_size:]
                    new_pred = predict_label_from_window(window)
                    print(f"[{datetime.now(timezone.utc)}] ML pred={new_pred}")
                    if new_pred != last_prediction:
                        center_price = float(fetch_latest_ohlcv(symbol, price_timeframe, 2)['close'].values[-1])

                        # If model predicts down (0), freeze and stop grids
                        if new_pred == 0:
                            print("[⚠️ PREDICTION DOWN] -> Closing ALL positions and stopping trading.")
                            last_price = float(fetch_latest_ohlcv(symbol, price_timeframe, 2)['close'].values[-1])
                            close_all_positions(last_price)
                            print(f"[BOT PAUSED] All positions closed. Balance={balance:.2f}. Waiting until model predicts HOLD or UP...")

                        # If model predicts hold (1) or up (2), continue grid trading
                        elif new_pred in (1, 2):
                            if last_prediction == 0:
                                print("[RESUME] Model predicts HOLD/UP -> Resuming trading.")
                            begin_graceful_transition(new_pred, center_price)

                        last_prediction = new_pred

                    else:
                        print("[PREDICT] unchanged")
                last_prediction_time = datetime.now(timezone.utc)

            # inner execution until next prediction
            inner_deadline = datetime.now(timezone.utc) + timedelta(seconds=prediction_interval)
            while datetime.now(timezone.utc) < inner_deadline:
                price_df = fetch_latest_ohlcv(symbol, price_timeframe, 2)
                last_price = float(price_df['close'].values[-1])
                
                # Get current ML prediction for this window
                feat_df = compute_indicators(fetch_latest_ohlcv(symbol, model_timeframe, limit)).dropna()
                if len(feat_df) >= window_size:
                    window = feat_df[FEATURES].values[-window_size:]
                    current_pred = predict_label_from_window(window)
                else:
                    current_pred = last_prediction

                # ----------------------------
                # If ML predicts DOWN → cancel everything
                # ----------------------------
                if current_pred == 0 or last_prediction == 0:
                    print("[⚠️ PREDICTION DOWN] -> Closing ALL positions and stopping trading.")
                    close_all_positions(last_price)  # remove positions and add to balance
                    for gid in list(grids.keys()):
                        cancel_grid_orders_ccxt(gid)  # cancel open orders
                        grids.pop(gid, None)
                    print(f"[BOT PAUSED] All positions closed. Balance={balance:.2f}. prediction is {current_pred} Waiting for bullish reversal...")
                    last_prediction = 0
                    continue  # keep inner loop paused

                last_prediction = current_pred

                # ----------------------------
                # Existing inner loop code
                # ----------------------------
                atr_now = get_current_atr()
                try_update_order_statuses()
                closed = try_resolve_positions_by_tp_sl(last_price)
                filled = try_fill_orders_sim_or_real(last_price)
                check_frozen_cutoffs(last_price)

                # Cleanup empty grids
                for gid, meta in list(grids.items()):
                    if not meta.get('frozen', False):
                        orders = get_open_orders_for_grid(gid)
                        has_pos = any(p['grid_id']==gid for p in open_positions)
                        if len(orders) == 0 and not has_pos:
                            print(f"[CLEANUP] active grid {gid} cleared (no exposure).")
                            grids.pop(gid, None)

                # Safety exposure & place missing grids
                exposure = total_open_exposure()
                exposure_pct = exposure / max(1e-8, balance)
                allow_place = exposure_pct <= max_total_exposure
                active_dirs = [meta['direction'] for gid, meta in grids.items() if not meta.get('frozen', False)]
                desired_dir = 'up' if last_prediction==2 else 'hold'
                if desired_dir not in active_dirs and allow_place:
                    place_grid(desired_dir, last_price, mode='trend')
                
                # Print status
                final_balance_now, profit_now = compute_realistic_profit(last_price)
                print(f"[STATUS] balance={balance:.2f} (simulated if stopped now: {final_balance_now:.2f}, profit={profit_now:.2f}), exposure={exposure:.2f} ({exposure_pct*100:.1f}%), grids={len(grids)}")
                
                
                print(f"[{datetime.now(timezone.utc)}] price={last_price:.2f} pred={last_prediction} bal={balance:.2f} pos={len(open_positions)} open_orders={len([o for o in open_orders if o['status']=='open'])} grids={len(grids)}")
                print("------------------------------------------------------------")
                time.sleep(check_interval_seconds)


    except KeyboardInterrupt:
        print("Stopped by user. Balance:", balance)
        print("Open positions:", open_positions)
        print("Open orders:", [o for o in open_orders if o['status']=='open'])

# -------------------------
# Run
# -------------------------
if __name__ == "__main__":
    run_bot()

[INIT] last_pred=2 (UP), price=110243.60
[INIT STATUS] Starting grid trading in UP mode.


/tmp/ipykernel_17769/3172839887.py:17: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



[GRID PLACED] id=0e37726c, dir=up, mode=trend, levels=17
------------------------------------------------------------
[DEBUG] Grid Details for 0e37726c:
Grid 1/17 | Buy: 109911.34 | TP: 109952.87 | SL: 109662.14
Grid 2/17 | Buy: 109952.87 | TP: 109994.40 | SL: 109703.68
Grid 3/17 | Buy: 109994.40 | TP: 110035.94 | SL: 109745.21
Grid 4/17 | Buy: 110035.94 | TP: 110077.47 | SL: 109786.74
Grid 5/17 | Buy: 110077.47 | TP: 110119.00 | SL: 109828.27
Grid 6/17 | Buy: 110119.00 | TP: 110160.53 | SL: 109869.81
Grid 7/17 | Buy: 110160.53 | TP: 110202.07 | SL: 109911.34
Grid 8/17 | Buy: 110202.07 | TP: 110243.60 | SL: 109952.87
Grid 9/17 | Buy: 110243.60 | TP: 110285.13 | SL: 109994.40
Grid 10/17 | Buy: 110285.13 | TP: 110326.67 | SL: 110035.94
Grid 11/17 | Buy: 110326.67 | TP: 110368.20 | SL: 110077.47
Grid 12/17 | Buy: 110368.20 | TP: 110409.73 | SL: 110119.00
Grid 13/17 | Buy: 110409.73 | TP: 110451.26 | SL: 110160.53
Grid 14/17 | Buy: 110451.26 | TP: 110492.80 | SL: 110202.07
Grid 15/17 | Buy